# Module Objects

object: module

> single: ModuleType (in module types)
>
> This instance of :c`PyTypeObject` represents the Python module type.
> This is exposed to Python programs as `types.ModuleType`.

> Return true if *p* is a module object, or a subtype of a module
> object. This function always succeeds.

> Return true if *p* is a module object, but not a subtype of
> :c`PyModule_Type`. This function always succeeds.

> single: \_\_name\_\_ (module attribute) single: \_\_doc\_\_ (module
> attribute) single: \_\_file\_\_ (module attribute) single:
> \_\_package\_\_ (module attribute) single: \_\_loader\_\_ (module
> attribute)
>
> Return a new module object with the `__name__` attribute set to
> *name*. The module's `__name__`, `__doc__`, `__package__`, and
> `__loader__` attributes are filled in (all but `__name__` are set to
> `None`); the caller is responsible for providing a `__file__`
> attribute.
>
> 3.3
>
> 3.4 `__package__` and `__loader__` are set to `None`.

> Similar to :c`PyModule_NewObject`, but the name is a UTF-8 encoded
> string instead of a Unicode object.

> single: \_\_dict\_\_ (module attribute)
>
> Return the dictionary object that implements *module*'s namespace;
> this object is the same as the `~object.__dict__` attribute of the
> module object. If *module* is not a module object (or a subtype of a
> module object), `SystemError` is raised and `NULL` is returned.
>
> It is recommended extensions use other :c`PyModule_\*` and
> :c`PyObject_\*` functions rather than directly manipulate a module's
> `~object.__dict__`.

> single: \_\_name\_\_ (module attribute) single: SystemError (built-in
> exception)
>
> Return *module*'s `__name__` value. If the module does not provide
> one, or if it is not a string, `SystemError` is raised and `NULL` is
> returned.
>
> 3.3

> Similar to :c`PyModule_GetNameObject` but return the name encoded to
> `'utf-8'`.

> Return the "state" of the module, that is, a pointer to the block of
> memory allocated at module creation time, or `NULL`. See
> :c`PyModuleDef.m_size`.

> Return a pointer to the :c`PyModuleDef` struct from which the module
> was created, or `NULL` if the module wasn't created from a definition.

> single: \_\_file\_\_ (module attribute) single: SystemError (built-in
> exception)
>
> Return the name of the file from which *module* was loaded using
> *module*'s `__file__` attribute. If this is not defined, or if it is
> not a unicode string, raise `SystemError` and return `NULL`; otherwise
> return a reference to a Unicode object.
>
> 3.2

> Similar to :c`PyModule_GetFilenameObject` but return the filename
> encoded to 'utf-8'.
>
> 3.2 :c`PyModule_GetFilename` raises :c`UnicodeEncodeError` on
> unencodable filenames, use :c`PyModule_GetFilenameObject` instead.

## Initializing C modules

Modules objects are usually created from extension modules (shared
libraries which export an initialization function), or compiled-in
modules (where the initialization function is added using
:c`PyImport_AppendInittab`). See `building` or
`extending-with-embedding` for details.

The initialization function can either pass a module definition instance
to :c`PyModule_Create`, and return the resulting module object, or
request "multi-phase initialization" by returning the definition struct
itself.

> The module definition struct, which holds all information needed to
> create a module object. There is usually only one statically
> initialized variable of this type for each module.
>
> > Always initialize this member to `PyModuleDef_HEAD_INIT`.
>
> > Name for the new module.
>
> > Docstring for the module; usually a docstring variable created with
> > :c`PyDoc_STRVAR` is used.
>
> > Module state may be kept in a per-module memory area that can be
> > retrieved with :c`PyModule_GetState`, rather than in static globals.
> > This makes modules safe for use in multiple sub-interpreters.
> >
> > This memory area is allocated based on *m_size* on module creation,
> > and freed when the module object is deallocated, after the
> > :c`m_free` function has been called, if present.
> >
> > Setting `m_size` to `-1` means that the module does not support
> > sub-interpreters, because it has global state.
> >
> > Setting it to a non-negative value means that the module can be
> > re-initialized and specifies the additional amount of memory it
> > requires for its state. Non-negative `m_size` is required for
> > multi-phase initialization.
> >
> > See [PEP 3121](http://www.python.org/dev/peps/pep-3121/ "PEP 3121")
> > for more details.
>
> > A pointer to a table of module-level functions, described by
> > :c`PyMethodDef` values. Can be `NULL` if no functions are present.
>
> > An array of slot definitions for multi-phase initialization,
> > terminated by a `{0, NULL}` entry. When using single-phase
> > initialization, *m_slots* must be `NULL`.
> >
> > 3.5
> >
> > Prior to version 3.5, this member was always set to `NULL`, and was
> > defined as:
>
> > A traversal function to call during GC traversal of the module
> > object, or `NULL` if not needed.
> >
> > This function is not called if the module state was requested but is
> > not allocated yet. This is the case immediately after the module is
> > created and before the module is executed (:c`Py_mod_exec`
> > function). More precisely, this function is not called if :c`m_size`
> > is greater than 0 and the module state (as returned by
> > :c`PyModule_GetState`) is `NULL`.
> >
> > 3.9 No longer called before the module state is allocated.
>
> > A clear function to call during GC clearing of the module object, or
> > `NULL` if not needed.
> >
> > This function is not called if the module state was requested but is
> > not allocated yet. This is the case immediately after the module is
> > created and before the module is executed (:c`Py_mod_exec`
> > function). More precisely, this function is not called if :c`m_size`
> > is greater than 0 and the module state (as returned by
> > :c`PyModule_GetState`) is `NULL`.
> >
> > Like :c`PyTypeObject.tp_clear`, this function is not *always* called
> > before a module is deallocated. For example, when reference counting
> > is enough to determine that an object is no longer used, the cyclic
> > garbage collector is not involved and :c`~PyModuleDef.m_free` is
> > called directly.
> >
> > 3.9 No longer called before the module state is allocated.
>
> > A function to call during deallocation of the module object, or
> > `NULL` if not needed.
> >
> > This function is not called if the module state was requested but is
> > not allocated yet. This is the case immediately after the module is
> > created and before the module is executed (:c`Py_mod_exec`
> > function). More precisely, this function is not called if :c`m_size`
> > is greater than 0 and the module state (as returned by
> > :c`PyModule_GetState`) is `NULL`.
> >
> > 3.9 No longer called before the module state is allocated.

### Single-phase initialization

The module initialization function may create and return the module
object directly. This is referred to as "single-phase initialization",
and uses one of the following two module creation functions:

> Create a new module object, given the definition in *def*. This
> behaves like :c`PyModule_Create2` with *module_api_version* set to
> `PYTHON_API_VERSION`.

> Create a new module object, given the definition in *def*, assuming
> the API version *module_api_version*. If that version does not match
> the version of the running interpreter, a `RuntimeWarning` is emitted.
>
> Note
>
> Most uses of this function should be using :c`PyModule_Create`
> instead; only use this if you are sure you need it.

Before it is returned from in the initialization function, the resulting
module object is typically populated using functions like
:c`PyModule_AddObjectRef`.

### Multi-phase initialization

An alternate way to specify extensions is to request "multi-phase
initialization". Extension modules created this way behave more like
Python modules: the initialization is split between the *creation
phase*, when the module object is created, and the *execution phase*,
when it is populated. The distinction is similar to the :py`__new__` and
:py`__init__` methods of classes.

Unlike modules created using single-phase initialization, these modules
are not singletons: if the *sys.modules* entry is removed and the module
is re-imported, a new module object is created, and the old module is
subject to normal garbage collection -- as with Python modules. By
default, multiple modules created from the same definition should be
independent: changes to one should not affect the others. This means
that all state should be specific to the module object (using e.g. using
:c`PyModule_GetState`), or its contents (such as the module's `__dict__`
or individual classes created with :c`PyType_FromSpec`).

All modules created using multi-phase initialization are expected to
support `sub-interpreters <sub-interpreter-support>`. Making sure
multiple modules are independent is typically enough to achieve this.

To request multi-phase initialization, the initialization function
(PyInit_modulename) returns a :c`PyModuleDef` instance with non-empty
:c`~PyModuleDef.m_slots`. Before it is returned, the `PyModuleDef`
instance must be initialized with the following function:

> Ensures a module definition is a properly initialized Python object
> that correctly reports its type and reference count.
>
> Returns *def* cast to `PyObject*`, or `NULL` if an error occurred.
>
> 3.5

The *m_slots* member of the module definition must point to an array of
`PyModuleDef_Slot` structures:

> > A slot ID, chosen from the available values explained below.
>
> > Value of the slot, whose meaning depends on the slot ID.
>
> 3.5

The *m_slots* array must be terminated by a slot with id 0.

The available slot types are:

> Specifies a function that is called to create the module object
> itself. The *value* pointer of this slot must point to a function of
> the signature:
>
> The function receives a :py`~importlib.machinery.ModuleSpec` instance,
> as defined in
> [PEP 451](http://www.python.org/dev/peps/pep-0451/ "PEP 451"), and the
> module definition. It should return a new module object, or set an
> error and return `NULL`.
>
> This function should be kept minimal. In particular, it should not
> call arbitrary Python code, as trying to import the same module again
> may result in an infinite loop.
>
> Multiple `Py_mod_create` slots may not be specified in one module
> definition.
>
> If `Py_mod_create` is not specified, the import machinery will create
> a normal module object using :c`PyModule_New`. The name is taken from
> *spec*, not the definition, to allow extension modules to dynamically
> adjust to their place in the module hierarchy and be imported under
> different names through symlinks, all while sharing a single module
> definition.
>
> There is no requirement for the returned object to be an instance of
> :c`PyModule_Type`. Any type can be used, as long as it supports
> setting and getting import-related attributes. However, only
> `PyModule_Type` instances may be returned if the `PyModuleDef` has
> non-`NULL` `m_traverse`, `m_clear`, `m_free`; non-zero `m_size`; or
> slots other than `Py_mod_create`.

> Specifies a function that is called to *execute* the module. This is
> equivalent to executing the code of a Python module: typically, this
> function adds classes and constants to the module. The signature of
> the function is:
>
> If multiple `Py_mod_exec` slots are specified, they are processed in
> the order they appear in the *m_slots* array.

See [PEP 489](http://www.python.org/dev/peps/pep-0489/ "PEP 489") for
more details on multi-phase initialization.

### Low-level module creation functions

The following functions are called under the hood when using multi-phase
initialization. They can be used directly, for example when creating
module objects dynamically. Note that both `PyModule_FromDefAndSpec` and
`PyModule_ExecDef` must be called to fully initialize a module.

> Create a new module object, given the definition in *module* and the
> ModuleSpec *spec*. This behaves like :c`PyModule_FromDefAndSpec2` with
> *module_api_version* set to `PYTHON_API_VERSION`.
>
> 3.5

> Create a new module object, given the definition in *module* and the
> ModuleSpec *spec*, assuming the API version *module_api_version*. If
> that version does not match the version of the running interpreter, a
> `RuntimeWarning` is emitted.
>
> Note
>
> Most uses of this function should be using :c`PyModule_FromDefAndSpec`
> instead; only use this if you are sure you need it.
>
> 3.5

> Process any execution slots (:c`Py_mod_exec`) given in *def*.
>
> 3.5

> Set the docstring for *module* to *docstring*. This function is called
> automatically when creating a module from `PyModuleDef`, using either
> `PyModule_Create` or `PyModule_FromDefAndSpec`.
>
> 3.5

> Add the functions from the `NULL` terminated *functions* array to
> *module*. Refer to the :c`PyMethodDef` documentation for details on
> individual entries (due to the lack of a shared module namespace,
> module level "functions" implemented in C typically receive the module
> as their first parameter, making them similar to instance methods on
> Python classes). This function is called automatically when creating a
> module from `PyModuleDef`, using either `PyModule_Create` or
> `PyModule_FromDefAndSpec`.
>
> 3.5

### Support functions

The module initialization function (if using single phase
initialization) or a function called from a module execution slot (if
using multi-phase initialization), can use the following functions to
help initialize the module state:

> Add an object to *module* as *name*. This is a convenience function
> which can be used from the module's initialization function.
>
> On success, return `0`. On error, raise an exception and return `-1`.
>
> Return `NULL` if *value* is `NULL`. It must be called with an
> exception raised in this case.
>
> Example usage:
>
> ``` c
> static int
> add_spam(PyObject *module, int value)
> {
>     PyObject *obj = PyLong_FromLong(value);
>     if (obj == NULL) {
>         return -1;
>     }
>     int res = PyModule_AddObjectRef(module, "spam", obj);
>     Py_DECREF(obj);
>     return res;
>  }
> ```
>
> The example can also be written without checking explicitly if *obj*
> is `NULL`:
>
> ``` c
> static int
> add_spam(PyObject *module, int value)
> {
>     PyObject *obj = PyLong_FromLong(value);
>     int res = PyModule_AddObjectRef(module, "spam", obj);
>     Py_XDECREF(obj);
>     return res;
>  }
> ```
>
> Note that `Py_XDECREF()` should be used instead of `Py_DECREF()` in
> this case, since *obj* can be `NULL`.
>
> 3.10

> Similar to :c`PyModule_AddObjectRef`, but steals a reference to
> *value* on success (if it returns `0`).
>
> The new :c`PyModule_AddObjectRef` function is recommended, since it is
> easy to introduce reference leaks by misusing the
> :c`PyModule_AddObject` function.
>
> Note
>
> Unlike other functions that steal references, `PyModule_AddObject()`
> only decrements the reference count of *value* **on success**.
>
> This means that its return value must be checked, and calling code
> must :c`Py_DECREF` *value* manually on error.
>
> Example usage:
>
> ``` c
> static int
> add_spam(PyObject *module, int value)
> {
>     PyObject *obj = PyLong_FromLong(value);
>     if (obj == NULL) {
>         return -1;
>     }
>     if (PyModule_AddObject(module, "spam", obj) < 0) {
>         Py_DECREF(obj);
>         return -1;
>     }
>     // PyModule_AddObject() stole a reference to obj:
>     // Py_DECREF(obj) is not needed here
>     return 0;
> }
> ```
>
> The example can also be written without checking explicitly if *obj*
> is `NULL`:
>
> ``` c
> static int
> add_spam(PyObject *module, int value)
> {
>     PyObject *obj = PyLong_FromLong(value);
>     if (PyModule_AddObject(module, "spam", obj) < 0) {
>         Py_XDECREF(obj);
>         return -1;
>     }
>     // PyModule_AddObject() stole a reference to obj:
>     // Py_DECREF(obj) is not needed here
>     return 0;
> }
> ```
>
> Note that `Py_XDECREF()` should be used instead of `Py_DECREF()` in
> this case, since *obj* can be `NULL`.

> Add an integer constant to *module* as *name*. This convenience
> function can be used from the module's initialization function. Return
> `-1` on error, `0` on success.

> Add a string constant to *module* as *name*. This convenience function
> can be used from the module's initialization function. The string
> *value* must be `NULL`-terminated. Return `-1` on error, `0` on
> success.

> Add an int constant to *module*. The name and the value are taken from
> *macro*. For example `PyModule_AddIntMacro(module, AF_INET)` adds the
> int constant *AF_INET* with the value of *AF_INET* to *module*. Return
> `-1` on error, `0` on success.

> Add a string constant to *module*.

> Add a type object to *module*. The type object is finalized by calling
> internally :c`PyType_Ready`. The name of the type object is taken from
> the last component of :c`~PyTypeObject.tp_name` after dot. Return `-1`
> on error, `0` on success.
>
> 3.9

## Module lookup

Single-phase initialization creates singleton modules that can be looked
up in the context of the current interpreter. This allows the module
object to be retrieved later with only a reference to the module
definition.

These functions will not work on modules created using multi-phase
initialization, since multiple such modules can be created from a single
definition.

> Returns the module object that was created from *def* for the current
> interpreter. This method requires that the module object has been
> attached to the interpreter state with :c`PyState_AddModule`
> beforehand. In case the corresponding module object is not found or
> has not been attached to the interpreter state yet, it returns `NULL`.

> Attaches the module object passed to the function to the interpreter
> state. This allows the module object to be accessible via
> :c`PyState_FindModule`.
>
> Only effective on modules created using single-phase initialization.
>
> Python calls `PyState_AddModule` automatically after importing a
> module, so it is unnecessary (but harmless) to call it from module
> initialization code. An explicit call is needed only if the module's
> own init code subsequently calls `PyState_FindModule`. The function is
> mainly intended for implementing alternative import mechanisms (either
> by calling it directly, or by referring to its implementation for
> details of the required state updates).
>
> The caller must hold the GIL.
>
> Return 0 on success or -1 on failure.
>
> 3.3

> Removes the module object created from *def* from the interpreter
> state. Return 0 on success or -1 on failure.
>
> The caller must hold the GIL.
>
> 3.3